# 데이터 전처리 및 피처 엔지니어링

In [0]:
root= '/FileStore/tables/bin'
retail = root+'/retail-data/by-day/*.csv'
simple_ml_int = root+'/simple-ml-integers'
simple_ml = root+'/simple-ml'
simple_ml_scale = root+'/simple-ml-scaling'

In [0]:
sales = spark.read.format('csv')\
.option('header', 'true')\
.option('inferSchema', 'true')\
.load(retail)\
.coalesce(5)\
.where("Description is not null")

In [0]:
fakeIntDF = spark.read.parquet(simple_ml_int)
simpleDF = spark.read.format('json').load(simple_ml)
scaleDF = spark.read.parquet(simple_ml_scale)

In [0]:
display(fakeIntDF)

int1,int2,int3
1,2,3
4,5,6
7,8,9


In [0]:
display(simpleDF.head(10))

color,lab,value1,value2
green,good,1,14.386294994851129
blue,bad,8,14.386294994851129
blue,bad,12,14.386294994851129
green,good,15,38.97187133755819
green,good,12,14.386294994851129
green,bad,16,14.386294994851129
red,good,35,14.386294994851129
red,bad,1,38.97187133755819
red,bad,2,14.386294994851129
red,bad,16,14.386294994851129


In [0]:
display(scaleDF)

id,features
0,"List(1, 3, List(), List(1.0, 0.1, -1.0))"
1,"List(1, 3, List(), List(2.0, 1.1, 1.0))"
0,"List(1, 3, List(), List(1.0, 0.1, -1.0))"
1,"List(1, 3, List(), List(2.0, 1.1, 1.0))"
1,"List(1, 3, List(), List(3.0, 10.1, 3.0))"


In [0]:
sales.cache()
display(sales.head(10))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
579256,20973,12 PENCIL SMALL TUBE WOODLAND,24,2011-11-29 08:07:00,0.65,13349.0,United Kingdom
579256,21891,TRADITIONAL WOODEN SKIPPING ROPE,12,2011-11-29 08:07:00,1.45,13349.0,United Kingdom
579256,22398,MAGNETS PACK OF 4 SWALLOWS,12,2011-11-29 08:07:00,0.39,13349.0,United Kingdom
579256,21746,SMALL RED RETROSPOT WINDMILL,12,2011-11-29 08:07:00,1.25,13349.0,United Kingdom
579256,21747,SMALL SKULL WINDMILL,12,2011-11-29 08:07:00,1.25,13349.0,United Kingdom
579256,22585,PACK OF 6 BIRDY GIFT TAGS,12,2011-11-29 08:07:00,1.25,13349.0,United Kingdom
579256,84228,HEN HOUSE WITH CHICK STANDING,12,2011-11-29 08:07:00,0.42,13349.0,United Kingdom
579256,21894,POTTING SHED SEED ENVELOPES,12,2011-11-29 08:07:00,1.25,13349.0,United Kingdom
579256,23433,HANGING QUILTED PATCHWORK APPLES,12,2011-11-29 08:07:00,0.83,13349.0,United Kingdom
579256,23476,WOODLAND LARGE RED FELT HEART,12,2011-11-29 08:07:00,1.25,13349.0,United Kingdom


## 텍스트 데이터 변환자
- 텍스트는 처리하기 매우 까다로운 데이터임
  - 머신러닝 모델이 효과적으로 작동하기 위해선 텍스트 데이터를 올바른 형식으로 매핑해야 하는데, 이는 많은 조작이 필요하기 때문
- 보통 텍스트 데이터는 자유형 텍스트와 문자열로된 범주형 변수가 있는데 앞에서 범주형을 다뤘으므로, 자유형을 중점적으로 볼 것임

### 텍스트 토큰화하기
- 자유형 텍스트를 '토큰' 또는 개별 단어 목록으로 변환하는 프로세스
- Tokenizer: 단어 컬럼을 공백으로 구분하여 단어 배열로 변환
- RegexTokenizer: 공백뿐만 아니라 정규 표현식ㅇ르 이용한 Tokenizer
  - Java의 정규 표현식 구문을 준수해야함

In [0]:
from pyspark.ml.feature import Tokenizer

tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized= tkn.transform(sales.select("Description").limit(10))
display(tokenized)

Description,DescOut
12 PENCIL SMALL TUBE WOODLAND,"List(12, pencil, small, tube, woodland)"
TRADITIONAL WOODEN SKIPPING ROPE,"List(traditional, wooden, skipping, rope)"
MAGNETS PACK OF 4 SWALLOWS,"List(magnets, pack, of, 4, swallows)"
SMALL RED RETROSPOT WINDMILL,"List(small, red, retrospot, windmill)"
SMALL SKULL WINDMILL,"List(small, skull, windmill)"
PACK OF 6 BIRDY GIFT TAGS,"List(pack, of, 6, birdy, gift, tags)"
HEN HOUSE WITH CHICK STANDING,"List(hen, house, with, chick, standing)"
POTTING SHED SEED ENVELOPES,"List(potting, shed, seed, envelopes)"
HANGING QUILTED PATCHWORK APPLES,"List(hanging, quilted, patchwork, apples)"
WOODLAND LARGE RED FELT HEART,"List(woodland, large, red, felt, heart)"


In [0]:
from pyspark.ml.feature import RegexTokenizer
rt = RegexTokenizer().setInputCol("Description").setOutputCol("DescOut").setPattern(" ").setToLowercase(True)
display(rt.transform(sales.select("Description").limit(10)))

Description,DescOut
12 PENCIL SMALL TUBE WOODLAND,"List(12, pencil, small, tube, woodland)"
TRADITIONAL WOODEN SKIPPING ROPE,"List(traditional, wooden, skipping, rope)"
MAGNETS PACK OF 4 SWALLOWS,"List(magnets, pack, of, 4, swallows)"
SMALL RED RETROSPOT WINDMILL,"List(small, red, retrospot, windmill)"
SMALL SKULL WINDMILL,"List(small, skull, windmill)"
PACK OF 6 BIRDY GIFT TAGS,"List(pack, of, 6, birdy, gift, tags)"
HEN HOUSE WITH CHICK STANDING,"List(hen, house, with, chick, standing)"
POTTING SHED SEED ENVELOPES,"List(potting, shed, seed, envelopes)"
HANGING QUILTED PATCHWORK APPLES,"List(hanging, quilted, patchwork, apples)"
WOODLAND LARGE RED FELT HEART,"List(woodland, large, red, felt, heart)"


In [0]:
#사전에 제시된 패턴에 매칭하는 값을 출력하도록(예제는 공백을 패턴으로 개별 단어를 포착하도록)
rt = RegexTokenizer().setInputCol("Description").setOutputCol("DescOut").setPattern(" ").setGaps(False).setToLowercase(True)
display(rt.transform(sales.select("Description").limit(10)))

Description,DescOut
12 PENCIL SMALL TUBE WOODLAND,"List( , , , )"
TRADITIONAL WOODEN SKIPPING ROPE,"List( , , )"
MAGNETS PACK OF 4 SWALLOWS,"List( , , , )"
SMALL RED RETROSPOT WINDMILL,"List( , , )"
SMALL SKULL WINDMILL,"List( , )"
PACK OF 6 BIRDY GIFT TAGS,"List( , , , , )"
HEN HOUSE WITH CHICK STANDING,"List( , , , )"
POTTING SHED SEED ENVELOPES,"List( , , )"
HANGING QUILTED PATCHWORK APPLES,"List( , , )"
WOODLAND LARGE RED FELT HEART,"List( , , , )"


### 일반적인 단어 제거하기
- 텍스트를 토큰화한 다음 일반적으로 불용어나 분석과 관련 없는 무의미한 용어를 필터링함
- 불용어 사용자 정의 가능
  - [doc](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.StopWordsRemover.html?highlight=stopwordsremover#pyspark.ml.feature.StopWordsRemover)

In [0]:
from pyspark.ml.feature import StopWordsRemover
#기본적으로 정의된 불용어 목록 로드
en_stopwords = StopWordsRemover.loadDefaultStopWords("english")
stops = StopWordsRemover().setStopWords(en_stopwords).setInputCol("DescOut")
display(stops.transform(tokenized))

Description,DescOut,StopWordsRemover_c6cf692b68c5__output
12 PENCIL SMALL TUBE WOODLAND,"List(12, pencil, small, tube, woodland)","List(12, pencil, small, tube, woodland)"
TRADITIONAL WOODEN SKIPPING ROPE,"List(traditional, wooden, skipping, rope)","List(traditional, wooden, skipping, rope)"
MAGNETS PACK OF 4 SWALLOWS,"List(magnets, pack, of, 4, swallows)","List(magnets, pack, 4, swallows)"
SMALL RED RETROSPOT WINDMILL,"List(small, red, retrospot, windmill)","List(small, red, retrospot, windmill)"
SMALL SKULL WINDMILL,"List(small, skull, windmill)","List(small, skull, windmill)"
PACK OF 6 BIRDY GIFT TAGS,"List(pack, of, 6, birdy, gift, tags)","List(pack, 6, birdy, gift, tags)"
HEN HOUSE WITH CHICK STANDING,"List(hen, house, with, chick, standing)","List(hen, house, chick, standing)"
POTTING SHED SEED ENVELOPES,"List(potting, shed, seed, envelopes)","List(potting, shed, seed, envelopes)"
HANGING QUILTED PATCHWORK APPLES,"List(hanging, quilted, patchwork, apples)","List(hanging, quilted, patchwork, apples)"
WOODLAND LARGE RED FELT HEART,"List(woodland, large, red, felt, heart)","List(woodland, large, red, felt, heart)"


In [0]:
#불용어 리스트
en_stopwords[:10]

Out[52]: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

### 단어 조합 만들기
 - 단어 조합이란 기술적으로 길이가 n인 단어의 시퀀스 즉, n-gram으로 간주됨
   - 길이가 1이면 unigram
   - 2면 bigram
   - 3이면 trigram
- n-gram을 생성하는 목적은 <strong>문장의 구조와 정보</strong>를 기존의 모든 단어를 개별적으로 살펴보는 것보다 더 잘 포착하기 위해서
  - 공통적으로 발생하는 단어 시퀀스를 파악할 수 있음 -> 단어 개별적으로 보는 것보다 더 나은 특징을 생성할 수 있음
- n-gram을 생성할 땐 <strong>순서</strong>가 중요함 왜냐면 3개의 단어로 구성된 문장을 bigram으로 표현하면 두 개의 bigram이 생성되기 때문

In [0]:
from pyspark.ml.feature import NGram
unigram = NGram().setInputCol("DescOut").setN(1)
bigram = NGram().setInputCol("DescOut").setN(2)
trigram = NGram().setInputCol("DescOut").setN(3)

In [0]:
target_df = tokenized.select("DescOut")
display(unigram.transform(target_df))

DescOut,NGram_5e73091ef002__output
"List(12, pencil, small, tube, woodland)","List(12, pencil, small, tube, woodland)"
"List(traditional, wooden, skipping, rope)","List(traditional, wooden, skipping, rope)"
"List(magnets, pack, of, 4, swallows)","List(magnets, pack, of, 4, swallows)"
"List(small, red, retrospot, windmill)","List(small, red, retrospot, windmill)"
"List(small, skull, windmill)","List(small, skull, windmill)"
"List(pack, of, 6, birdy, gift, tags)","List(pack, of, 6, birdy, gift, tags)"
"List(hen, house, with, chick, standing)","List(hen, house, with, chick, standing)"
"List(potting, shed, seed, envelopes)","List(potting, shed, seed, envelopes)"
"List(hanging, quilted, patchwork, apples)","List(hanging, quilted, patchwork, apples)"
"List(woodland, large, red, felt, heart)","List(woodland, large, red, felt, heart)"


In [0]:
display(bigram.transform(target_df))

DescOut,NGram_77257c00b7e7__output
"List(12, pencil, small, tube, woodland)","List(12 pencil, pencil small, small tube, tube woodland)"
"List(traditional, wooden, skipping, rope)","List(traditional wooden, wooden skipping, skipping rope)"
"List(magnets, pack, of, 4, swallows)","List(magnets pack, pack of, of 4, 4 swallows)"
"List(small, red, retrospot, windmill)","List(small red, red retrospot, retrospot windmill)"
"List(small, skull, windmill)","List(small skull, skull windmill)"
"List(pack, of, 6, birdy, gift, tags)","List(pack of, of 6, 6 birdy, birdy gift, gift tags)"
"List(hen, house, with, chick, standing)","List(hen house, house with, with chick, chick standing)"
"List(potting, shed, seed, envelopes)","List(potting shed, shed seed, seed envelopes)"
"List(hanging, quilted, patchwork, apples)","List(hanging quilted, quilted patchwork, patchwork apples)"
"List(woodland, large, red, felt, heart)","List(woodland large, large red, red felt, felt heart)"


In [0]:
display(trigram.transform(target_df))

DescOut,NGram_28cd522676b3__output
"List(12, pencil, small, tube, woodland)","List(12 pencil small, pencil small tube, small tube woodland)"
"List(traditional, wooden, skipping, rope)","List(traditional wooden skipping, wooden skipping rope)"
"List(magnets, pack, of, 4, swallows)","List(magnets pack of, pack of 4, of 4 swallows)"
"List(small, red, retrospot, windmill)","List(small red retrospot, red retrospot windmill)"
"List(small, skull, windmill)",List(small skull windmill)
"List(pack, of, 6, birdy, gift, tags)","List(pack of 6, of 6 birdy, 6 birdy gift, birdy gift tags)"
"List(hen, house, with, chick, standing)","List(hen house with, house with chick, with chick standing)"
"List(potting, shed, seed, envelopes)","List(potting shed seed, shed seed envelopes)"
"List(hanging, quilted, patchwork, apples)","List(hanging quilted patchwork, quilted patchwork apples)"
"List(woodland, large, red, felt, heart)","List(woodland large red, large red felt, red felt heart)"


### 단어를 숫자로 변환하기
- 단어 특징을 생성하려면 모델에서 사용하기 위해 단어와 단어 조합 수를 세어야함
  - 가장 간단한 방법: 주어진 문서에 단어 포함 여부를 바이너리 형식으로 세기

#### CounterVectorizer
  - 토큰화된 데이터에서 작동
  - 모델을 적합하는 프로세스 동안 모든 문서에서 단어 집합을 찾고 문서별로 해당 단어의 출현 빈도를 계산함
  - 변환 과정에서 Dataframe 컬럼의 각 로우(문서)에서 주어진 단어의 발생 빈도를 계산하고 해당 로우에 포함된 용어(단어)를 벡터 형태로 출력함
  - 이 변환자는 모든 <strong>로우를 문서</strong>로 취급 / 모든 <strong>단어를 용어</strong>로 취급 / 모든 <strong>용어의 집합을 어휘집</strong>으로 취급

In [0]:
from pyspark.ml.feature import CountVectorizer

#어휘집 최대크기 제한 가능 / 어휘집에 포함될 용어에 대한 최소 빈도 제한 가능(minTF) / 어휘집에 포함되기 전에 용어가 나타나야 하는 최소 문서 빈도 제한 가능(minDF)
cv = CountVectorizer().setInputCol("DescOut").setOutputCol("countVec").setVocabSize(500).setMinTF(1).setMinDF(2)
fittedCV = cv.fit(target_df)
display(fittedCV.transform(target_df))

DescOut,countVec
"List(12, pencil, small, tube, woodland)","List(0, 4, List(), List())"
"List(traditional, wooden, skipping, rope)","List(0, 4, List(), List())"
"List(magnets, pack, of, 4, swallows)","List(0, 4, List(1, 2), List(1.0, 1.0))"
"List(small, red, retrospot, windmill)","List(0, 4, List(), List())"
"List(small, skull, windmill)","List(0, 4, List(), List())"
"List(pack, of, 6, birdy, gift, tags)","List(0, 4, List(1), List(1.0))"
"List(hen, house, with, chick, standing)","List(0, 4, List(), List())"
"List(potting, shed, seed, envelopes)","List(0, 4, List(), List())"
"List(hanging, quilted, patchwork, apples)","List(0, 4, List(), List())"
"List(woodland, large, red, felt, heart)","List(0, 4, List(), List())"


#### TF-IDF
- Term Frequency - Inverse Document Frequency
- 얼마나 많은 문서가 그 용어를 포함하고 있는지에 따라 가중치를 부여
- 여러 문서에서 발생하는 용어보다 적은 문서에서 출현하는 용어에 더 많은 가중치 부여
  - 'the' 같이 일반적인 단어는 낮게 'streaming'같은 전문 용어는 높게
- TF-IDF는 비슷한 주제를 가진 문서를 찾는데도 사용할 수 있음

In [0]:
#red 단어가 포함된 일부 문서 추출
tfIdfIn = tkn.transform(sales).where("array_contains(DescOut, 'red')").select("DescOut")
display(tfIdfIn.limit(10))

DescOut
"List(small, red, retrospot, windmill)"
"List(woodland, large, red, felt, heart)"
"List(lunch, bag, red, vintage, doily)"
"List(red, floral, feltcraft, shoulder, bag)"
"List(feltcraft, hairband, red, and, blue)"
"List(jumbo, shopper, vintage, red, paisley)"
"List(vintage, doily, jumbo, bag, red)"
"List(red, retrospot, charlotte, bag)"
"List(jumbo, bag, red, retrospot)"
"List(vintage, red, enamel, trim, jug)"


In [0]:
#TF-IDF의 인풋은 각 단어를 해싱해서 수치형으로 변환한 데이터를 넣어줘야함
#HashingTF는 CountVeoctorizer와 비슷한 과정인데 결과를 되돌릴 수 없다는 점이 차이점(출력 색인으로부터 입력 단어 추적이 불가능하단말) + 여러 단어가 동일한 출력 색인에 매핑될 수 있음
from pyspark.ml.feature import HashingTF, IDF

tf = HashingTF().setInputCol("DescOut").setOutputCol("TFOut").setNumFeatures(10000)
idf =IDF().setInputCol("TFOut").setOutputCol("IDFOut").setMinDocFreq(2)
trans_tf = tf.transform(tfIdfIn)
display(idf.fit(trans_tf).transform(trans_tf).limit(10))

DescOut,TFOut,IDFOut
"List(small, red, retrospot, windmill)","List(0, 10000, List(52, 222, 6635, 8448), List(1.0, 1.0, 1.0, 1.0))","List(0, 10000, List(52, 222, 6635, 8448), List(0.0, 3.4420193761824107, 4.6858136746950185, 0.749707998750915))"
"List(woodland, large, red, felt, heart)","List(0, 10000, List(52, 4686, 7022, 9673, 9808), List(1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 10000, List(52, 4686, 7022, 9673, 9808), List(0.0, 6.1366465569524795, 4.35606038832255, 6.1366465569524795, 2.9853360515800453))"
"List(lunch, bag, red, vintage, doily)","List(0, 10000, List(52, 415, 2056, 2824, 6801), List(1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 10000, List(52, 415, 2056, 2824, 6801), List(0.0, 1.629089199831389, 2.5284350059059983, 4.306735321760049, 3.173093181762724))"
"List(red, floral, feltcraft, shoulder, bag)","List(0, 10000, List(50, 52, 415, 6756, 8005), List(1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 10000, List(50, 52, 415, 6756, 8005), List(4.500891336201006, 0.0, 1.629089199831389, 4.922723421773376, 4.922723421773376))"
"List(feltcraft, hairband, red, and, blue)","List(0, 10000, List(50, 52, 2891, 5768, 9501), List(1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 10000, List(50, 52, 2891, 5768, 9501), List(4.500891336201006, 0.0, 4.444093737807873, 4.846496056385492, 5.548859892050361))"
"List(jumbo, shopper, vintage, red, paisley)","List(0, 10000, List(52, 322, 1076, 2056, 9104), List(1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 10000, List(52, 322, 1076, 2056, 9104), List(0.0, 3.202789687116576, 3.2859400554487466, 2.5284350059059983, 2.464574221154925))"
"List(vintage, doily, jumbo, bag, red)","List(0, 10000, List(52, 415, 2056, 2824, 9104), List(1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 10000, List(52, 415, 2056, 2824, 9104), List(0.0, 1.629089199831389, 2.5284350059059983, 4.306735321760049, 2.464574221154925))"
"List(red, retrospot, charlotte, bag)","List(0, 10000, List(52, 415, 4175, 8448), List(1.0, 1.0, 1.0, 1.0))","List(0, 10000, List(52, 415, 4175, 8448), List(0.0, 1.629089199831389, 3.662913207762605, 0.749707998750915))"
"List(jumbo, bag, red, retrospot)","List(0, 10000, List(52, 415, 8448, 9104), List(1.0, 1.0, 1.0, 1.0))","List(0, 10000, List(52, 415, 8448, 9104), List(0.0, 1.629089199831389, 0.749707998750915, 2.464574221154925))"
"List(vintage, red, enamel, trim, jug)","List(0, 10000, List(52, 2056, 5333, 9980, 9989), List(1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 10000, List(52, 2056, 5333, 9980, 9989), List(0.0, 2.5284350059059983, 4.402045501564373, 4.396180382111975, 4.431898464714054))"


-----
- IDFOut컬럼의 밀집 벡터를 보면 red는 매우 낮은 가중치를 갖는다는 것을 알 수 잇음
  - 벡터는 총 어휘 크기, 문서에 나타나는 모든 단어의 해시, 각 용어의 가중치 등 세 가지 다른 값을 사용하여 표현됨(CountVectorizer와 유사)

### Word2Vec
- 단어 집합의 벡터 표현을 계산하기 위한 딥러닝 기반 도구
- 목표는 비슷한 단어를 벡터 공간에서 서로 가깝게 배치하여 단어를 일반화할 수 있도록 하는 것
- 이 모델은 학습 및 사용이 쉽고, 엔티티 인식, 모호성 제거, 구문 분석, 태그 지정 및 기계 번역을 포함한 여러 가지 자연어 처리 애플리케이션에서 유용하게 활용 가능
- 의미에 따라 단어 간의 관계를 파악하는 데 특히 유용함

In [0]:
from pyspark.ml.feature import Word2Vec
documentDF = spark.createDataFrame([
  ('Hi, I heard about Spark'.split(' '),),
  ('I wish Java could use case classes'.split(' '),),
  ('Logistic regression models are neat'.split(' '),)
  
],['text'])

In [0]:
word2vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2vec.fit(documentDF)
result=model.transform(documentDF)
for row in result.collect():
  txt, vector= row
  print(f"TEXT:{', '.join(txt)} => \n VECTOR:{vector}\n")

TEXT:Hi,, I, heard, about, Spark => 
 VECTOR:[-0.03353322595357895,0.08199198935180903,-0.0053790161386132245]

TEXT:I, wish, Java, could, use, case, classes => 
 VECTOR:[-0.00478374760132283,-0.032776287091629844,-0.04977526156497853]

TEXT:Logistic, regression, models, are, neat => 
 VECTOR:[0.05995699986815453,-0.04438269957900048,0.021498356014490128]

## 특징 조작하기
- 대부분의 변환자는 다양한 방식으로 특징 공간을 조작함
- 앞으로 소개할 알고리즘 및 도구는 입력 특징 벡터를 확장하거나 더 작은 수의 차원으로 축소하는 자동화된 방법들

### 주성분 분석
- 주성분 분석(Principal Components Analysis, PCA)은 데이터의 가장 중요한 측면(주요 구성 요소)을 찾는 수학적 기법
  - 새로운 특징 집합을 작성해서 데이터의 특징 표현을 변경함
  - 각각의 새로운 특징은 기존 특징들의 조합임
- PCA는 <strong>데이터의 주요 정보를 포함하는 더 작은 특징 집합을 생성</strong>할 수 있다는 강점을 지님
  - 그래서 대규모 입력 데이터셋에서 총 특징 수를 줄이기 위해 사용함
  - 보통 전체 특징 공간이 방대하고 많은 특징이 큰 관련성이 없는 텍스트 분석에 용이

In [0]:
from pyspark.ml.feature import PCA
pca= PCA().setInputCol("features").setK(2)
display(pca.fit(scaleDF).transform(scaleDF))

id,features,PCA_63b51ad4ddfd__output
0,"List(1, 3, List(), List(1.0, 0.1, -1.0))","List(1, 2, List(), List(0.07137194992484175, -0.45266548881478375))"
1,"List(1, 3, List(), List(2.0, 1.1, 1.0))","List(1, 2, List(), List(-1.680494698407372, 1.2593401322219153))"
0,"List(1, 3, List(), List(1.0, 0.1, -1.0))","List(1, 2, List(), List(0.07137194992484175, -0.45266548881478375))"
1,"List(1, 3, List(), List(2.0, 1.1, 1.0))","List(1, 2, List(), List(-1.680494698407372, 1.2593401322219153))"
1,"List(1, 3, List(), List(3.0, 10.1, 3.0))","List(1, 2, List(), List(-10.872398139848944, 0.030962697060150646))"


### 상호작용
- 사용자가 데이터 집합의 특정 변수에 대한 도메인 지식을 가지고 있어서 두 변수 사이의 상호작용을 수동으로 만들 수 있음
- 파이썬은 RFormula로, 스파크는 Interaction 변환자도 사용 가능

### 다항식 전개
- 모든 입력 컬럼의 상호작용 변수를 생성하는 데 사용

In [0]:
from pyspark.ml.feature import PolynomialExpansion
#2차 다항식
pe= PolynomialExpansion().setInputCol("features").setDegree(2)
display(pe.transform(scaleDF))

id,features,PolynomialExpansion_02fa3fadb4b3__output
0,"List(1, 3, List(), List(1.0, 0.1, -1.0))","List(1, 9, List(), List(1.0, 1.0, 0.1, 0.1, 0.010000000000000002, -1.0, -1.0, -0.1, 1.0))"
1,"List(1, 3, List(), List(2.0, 1.1, 1.0))","List(1, 9, List(), List(2.0, 4.0, 1.1, 2.2, 1.2100000000000002, 1.0, 2.0, 1.1, 1.0))"
0,"List(1, 3, List(), List(1.0, 0.1, -1.0))","List(1, 9, List(), List(1.0, 1.0, 0.1, 0.1, 0.010000000000000002, -1.0, -1.0, -0.1, 1.0))"
1,"List(1, 3, List(), List(2.0, 1.1, 1.0))","List(1, 9, List(), List(2.0, 4.0, 1.1, 2.2, 1.2100000000000002, 1.0, 2.0, 1.1, 1.0))"
1,"List(1, 3, List(), List(3.0, 10.1, 3.0))","List(1, 9, List(), List(3.0, 9.0, 10.1, 30.299999999999997, 102.00999999999999, 3.0, 9.0, 30.299999999999997, 9.0))"


## 특징 선택
- 다양한 특징이 강한 상관관계를 나타내거나 너무 많은 특징을 학습에 사용하는 것은 과적합을 초래할 수 있다.
- 그래서 사전에 특징을 필터링하는 것이 좋다.

### ChiSqSelector
- 통계적 검정을 활용하여 예측하려는 레이블과 독립적이지 않은 특징을 식별하고 관련 없는 특징을 삭제함

In [0]:
from pyspark.ml.feature import ChiSqSelector, Tokenizer
tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn.transform(sales.select("Description", "CustomerId").where("CustomerId is not null"))
prechi = fittedCV.transform(tokenized).where("customerId is not null")
chisq = ChiSqSelector().setFeaturesCol("countVec").setLabelCol("CustomerId").setNumTopFeatures(2)

In [0]:
display(chisq.fit(prechi).transform(prechi).limit(10))

Description,CustomerId,DescOut,countVec,ChiSqSelector_e84c3165fe9a__output
12 PENCIL SMALL TUBE WOODLAND,13349.0,"List(12, pencil, small, tube, woodland)","List(0, 4, List(), List())","List(0, 2, List(), List())"
TRADITIONAL WOODEN SKIPPING ROPE,13349.0,"List(traditional, wooden, skipping, rope)","List(0, 4, List(), List())","List(0, 2, List(), List())"
MAGNETS PACK OF 4 SWALLOWS,13349.0,"List(magnets, pack, of, 4, swallows)","List(0, 4, List(1, 2), List(1.0, 1.0))","List(0, 2, List(1), List(1.0))"
SMALL RED RETROSPOT WINDMILL,13349.0,"List(small, red, retrospot, windmill)","List(0, 4, List(), List())","List(0, 2, List(), List())"
SMALL SKULL WINDMILL,13349.0,"List(small, skull, windmill)","List(0, 4, List(), List())","List(0, 2, List(), List())"
PACK OF 6 BIRDY GIFT TAGS,13349.0,"List(pack, of, 6, birdy, gift, tags)","List(0, 4, List(1), List(1.0))","List(0, 2, List(1), List(1.0))"
HEN HOUSE WITH CHICK STANDING,13349.0,"List(hen, house, with, chick, standing)","List(0, 4, List(), List())","List(0, 2, List(), List())"
POTTING SHED SEED ENVELOPES,13349.0,"List(potting, shed, seed, envelopes)","List(0, 4, List(), List())","List(0, 2, List(), List())"
HANGING QUILTED PATCHWORK APPLES,13349.0,"List(hanging, quilted, patchwork, apples)","List(0, 4, List(), List())","List(0, 2, List(), List())"
WOODLAND LARGE RED FELT HEART,13349.0,"List(woodland, large, red, felt, heart)","List(0, 4, List(), List())","List(0, 2, List(), List())"


## 고급 주제

### 변환자 저장하기
- write 메서드를 사용하여 디스크에 기록

In [0]:
root

Out[24]: '/FileStore/tables/bin'

In [0]:
fittedPCA = pca.fit(scaleDF)
fittedPCA.write().overwrite().save(root+'/fittedPCA')

In [0]:
#불러올땐 PCAModel로
from pyspark.ml.feature import PCAModel
loadedPCA =PCAModel.load(root+'/fittedPCA')
loadedPCA.transform(scaleDF).show()

+---+--------------+-----------------------------+
 id| features|PCAModel_995e39edc42c__output|
+---+--------------+-----------------------------+
 0|[1.0,0.1,-1.0]| [0.07137194992484...|
 1| [2.0,1.1,1.0]| [-1.6804946984073...|
 0|[1.0,0.1,-1.0]| [0.07137194992484...|
 1| [2.0,1.1,1.0]| [-1.6804946984073...|
 1|[3.0,10.1,3.0]| [-10.872398139848...|
+---+--------------+-----------------------------+

### 사용자 정의 변환자 작성하기
- 필요한 경우
  - ML파이프라인에 적합시키고, 하이퍼파라미터 검색에 전달할 수 있는 형식으로 자신의 비즈니스 논리 중 일부를 인코딩 하려는 경우
  - 실제 입력 데이터를 기반으로 변환자를 변형해야하는 경우